In [26]:
import pandas as pd

In [27]:
events_training = pd.read_csv('training.csv')
events_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')
feature_uno_training = pd.read_csv('feature_training_paso_uno.csv')
feature_uno_test = pd.read_csv('feature_test_paso_uno.csv')

# Primero agregamos features al set de entrenamiento

# Primero voy a agregar un feature que es la cantidad de tiempo promedio entre conversiones.

In [28]:
#agrego la columna date al set de datos
events_training['date'] = pd.to_datetime(events_training['timestamp'])

In [29]:
eventos = events_training.sort_values(by='date',ascending=True).drop('Unnamed: 0',axis=1)
count_conversion = eventos.loc[eventos['event']=='conversion',['person','event','timestamp']]\
            .groupby(['person','event']).count().reset_index()
count_conversion.columns=['person','event','count']
count_conversion_uno = count_conversion.loc[:,['person','count']].groupby('person')\
            .filter(lambda x : x['count'] == 1 ).reset_index(drop=True)
count_conversion_mas_uno = count_conversion.loc[:,['person','count']].groupby('person')\
            .filter(lambda x : x['count'] > 1 ).reset_index(drop=True)

In [30]:
datos_uno = eventos.merge(count_conversion_mas_uno,on='person',how='inner')

In [31]:
datos_uno = datos_uno.loc[datos_uno['event']=='conversion',['person','date']]

In [32]:
datos_uno['person_shift'] = datos_uno['person'].shift(periods=1,axis=0)
datos_uno['date_shift'] = datos_uno['date'].shift(periods=1,axis=0)
datos_uno['bool']=(datos_uno['person_shift']==datos_uno['person'])
datos_uno = datos_uno.loc[datos_uno['bool']==True,['person','date','date_shift']]

In [33]:
datos_uno['dist_time']=(datos_uno['date']-datos_uno['date_shift']).dt.total_seconds()
datos_uno = datos_uno.loc[:,['person','dist_time']]

In [34]:
datos_dos = count_conversion_uno
datos_dos['dist_time']=-1
datos_dos = datos_dos.loc[:,['person','dist_time']]
datos_dos.head()

,person,dist_time
0,001b0bf9,-1
1,0020152e,-1
2,004ed8ba,-1
3,00971729,-1
4,00ecfd1a,-1


In [35]:
datos_uno.head()

,person,dist_time
609,8af11dbc,2956579.0
615,8af11dbc,2702.0
808,dbf6b08a,45683.0
1217,40f9b318,11006649.0
1219,40f9b318,6300.0


In [36]:
datos_tres = eventos.groupby('person').filter(lambda x: (x['event']!='conversion').all()).reset_index(drop=True)
datos_tres = datos_tres.loc[:,['person']].drop_duplicates(keep='first')
datos_tres['dist_time']=-2
datos_tres.head()

,person,dist_time
0,53a01f1e,-2
16,d8001b23,-2
19,7a2b9102,-2
57,0d741016,-2
61,09252ee6,-2


In [37]:
time_mean = datos_uno.groupby('person').mean().reset_index()
datos_cuatro = time_mean.merge(datos_tres,on=['person','dist_time'],how='outer')\
            .merge(datos_dos,on=['person','dist_time'],how='outer')
datos_cuatro.columns=['person','dist_time_mean']

In [38]:
valores = feature_uno_training.merge(datos_cuatro,on='person',how='inner')

In [39]:
valores.head()

,person,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,dist_time_mean
0,0566e9c1,0,viewed product,23,checkout,1,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0,7.555556,-1.0
1,6ec7ee77,0,ad campaign hit,1,ad campaign hit,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,-2.0
2,abe7a2fb,0,viewed product,31,checkout,1,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0,12.000000,-2.0
3,34728364,0,viewed product,24,search engine hit,1,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,4.0,7.400000,-2.0
4,87ed62de,0,viewed product,9,visited site,1,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,4.250000,-2.0


# Repito toda la operacion para el set de entrenamiento.

In [40]:
#agrego la columna date al set de datos
events_test['date'] = pd.to_datetime(events_test['timestamp'])

In [41]:
eventos_test = events_test.sort_values(by='date',ascending=True).drop('Unnamed: 0',axis=1)
count_conversion = eventos_test.loc[eventos_test['event']=='conversion',['person','event','timestamp']]\
            .groupby(['person','event']).count().reset_index()
count_conversion.columns=['person','event','count']
count_conversion_uno = count_conversion.loc[:,['person','count']].groupby('person')\
            .filter(lambda x : x['count'] == 1 ).reset_index(drop=True)
count_conversion_mas_uno = count_conversion.loc[:,['person','count']].groupby('person')\
            .filter(lambda x : x['count'] > 1 ).reset_index(drop=True)

In [42]:
datos_uno = eventos_test.merge(count_conversion_mas_uno,on='person',how='inner')

In [43]:
datos_uno = datos_uno.loc[datos_uno['event']=='conversion',['person','date']]

In [44]:
datos_uno['person_shift'] = datos_uno['person'].shift(periods=1,axis=0)
datos_uno['date_shift'] = datos_uno['date'].shift(periods=1,axis=0)
datos_uno['bool']=(datos_uno['person_shift']==datos_uno['person'])
datos_uno = datos_uno.loc[datos_uno['bool']==True,['person','date','date_shift']]

In [45]:
datos_uno['dist_time']=(datos_uno['date']-datos_uno['date_shift']).dt.total_seconds()
datos_uno = datos_uno.loc[:,['person','dist_time']]

In [46]:
datos_dos = count_conversion_uno
datos_dos['dist_time']=-1
datos_dos = datos_dos.loc[:,['person','dist_time']]
datos_dos.head()

,person,dist_time
0,000ba417,-1
1,001001be,-1
2,001804a2,-1
3,0038a117,-1
4,00750af8,-1


In [47]:
datos_uno.head()

,person,dist_time
332,a66e8424,11717487.0
350,a66e8424,356534.0
990,25950776,5530600.0
991,25950776,158.0
1109,ee3d07df,134971.0


In [48]:
datos_tres = eventos_test.groupby('person').filter(lambda x: (x['event']!='conversion').all()).reset_index(drop=True)
datos_tres = datos_tres.loc[:,['person']].drop_duplicates(keep='first')
datos_tres['dist_time']=-2
datos_tres.head()

,person,dist_time
0,0f4e2a4b,-2
17,23252ece,-2
23,1a260aa4,-2
30,a957e529,-2
47,99841b84,-2


In [49]:
time_mean = datos_uno.groupby('person').mean().reset_index()
datos_cuatro = time_mean.merge(datos_tres,on=['person','dist_time'],how='outer')\
            .merge(datos_dos,on=['person','dist_time'],how='outer')
datos_cuatro.columns=['person','dist_time_mean']

In [51]:
valores_test = feature_uno_test.merge(datos_cuatro,on='person',how='inner')

In [52]:
valores_test.head()

,person,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,mean_per_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,dist_time_mean
0,4886f805,viewed product,4,checkout,1,1.500000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,-2.0
1,0297fc1e,viewed product,404,lead,1,70.875000,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0,-2.0
2,2d681dd8,viewed product,13,ad campaign hit,1,3.250000,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0,-2.0
3,cccea85e,viewed product,739,checkout,1,92.888889,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0,-2.0
4,4c8a8b93,viewed product,177,checkout,2,32.125000,14.0,8.0,2.0,0.0,14.0,0.0,13.0,9.0,0.0,177.0,20.0,-2.0


In [53]:
valores.to_csv('feature_training_paso_dos_promydev.csv',index=False)
valores_test.to_csv('feature_test_paso_dos_promydev.csv',index=False)